In [ ]:
import sys
sys.path.append('../src/')

import os
from os import PathLike
from pathlib import Path

import data_io

from aind_vr_foraging_analysis.utils.parsing import parse, data_access
import aind_vr_foraging_analysis.utils.plotting as plotting

# Plotting libraries
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import seaborn as sns
import pandas as pd
import numpy as np
from matplotlib.patches import Rectangle

def format_func(value, tick_number):
    return f"{value:.0f}"


sns.set_context('talk')

import warnings
pd.options.mode.chained_assignment = None  # Ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [ ]:
save_path = r"C:\Users\tiffany.ona\OneDrive - Allen Institute\Documents\VR foraging\experiments\choosing odors"

session = session_path[65:73]
session_path = Path(session_path)

In [ ]:
import harp

path_to_data = r"Z:\scratch\vr-foraging\olfactometer_calibration\calibration_batch5_03122025\OlfactometerCalibration_2025-03-14T004321Z\behavior" # Path to the data folder. This is where the .harp files are located

board = r"/Olfactometer.harp" # Change this to the board you want to read from. You can find the boards in the path_to_data folder

device = harp.create_reader(path_to_data + board) # create the data asset. Basically you have access this way to all the registers in the board


In [ ]:
def recover_session(session_path):
    session_path = Path(session_path)
    data = parse.load_session_data(session_path)
    
    data['harp_olfactometer'].streams.OdorValveState.load_from_file()
    data['harp_olfactometer'].streams.EndValveState.load_from_file()  
    data['harp_analog'].streams.AnalogData.load_from_file()
    data_analog = data['harp_analog'].streams.AnalogData.data['Channel0'].reset_index()
    data_analog.set_index('Time', inplace=True)

    # Recover end valve triggers
    endvalve = data['harp_olfactometer'].streams.EndValveState.data['EndValve0'].reset_index()
    assert(np.round(endvalve.Time.diff().median(),1))
    endvalve = endvalve.loc[endvalve.EndValve0 == 1]
    endvalve.rename(columns={'EndValve0': 'data'}, inplace=True)

    if 'Rawrig' in data['config'].streams:
        data['config'].streams.RawRig.load_from_file()
        rig = data['config'].streams.RawRig.data['rig_name']
        data['config'].streams.RawFullModel.load_from_file()
        odor_dict = {key: data['config'].streams.RawFullModel.data['operation_control']['stimulus_config'][key]['odorant'] for key in ['0', '1', '2']}
        duration = data['config'].streams.RawFullModel.data['operation_control']['time_on']

    else:
        data['config'].streams.rig_input.load_from_file()
        rig = data['config'].streams.rig_input.data['rig_name']
        data['config'].streams.tasklogic_input.load_from_file()
        odorant_data = data['config'].streams.tasklogic_input.data['task_parameters']['channel_config']
        odor_dict = {key: odorant_data[key]['odorant'] for key in odorant_data}
            
        duration = data['config'].streams.tasklogic_input.data['task_parameters']['time_on']

    print(f"Rig: {rig}")
    data['harp_olfactometer'].streams.Channel0ActualFlow.load_from_file()
    data['harp_olfactometer'].streams.Channel0TargetFlow.load_from_file()

    data['harp_olfactometer'].streams.Channel1ActualFlow.load_from_file()
    data['harp_olfactometer'].streams.Channel1TargetFlow.load_from_file()

    data['harp_olfactometer'].streams.Channel2ActualFlow.load_from_file()
    data['harp_olfactometer'].streams.Channel2TargetFlow.load_from_file()

    data['harp_olfactometer'].streams.Channel3ActualFlow.load_from_file()
    data['harp_olfactometer'].streams.Channel3TargetFlow.load_from_file()

    data['harp_olfactometer'].streams.Channel4ActualFlow.load_from_file()
    data['harp_olfactometer'].streams.Channel4TargetFlow.load_from_file()

    data['harp_olfactometer'].streams.Flowmeter.load_from_file()

    valve0 = data['harp_olfactometer'].streams.OdorValveState.data['Valve0'].reset_index()
    valve0.set_index('Time', inplace=True)
    valve0 = valve0.loc[valve0.Valve0 == 1]

    valve1 = data['harp_olfactometer'].streams.OdorValveState.data['Valve1'].reset_index()
    valve1.set_index('Time', inplace=True)
    valve1_close = valve1.index.diff()
    valve1 = valve1.loc[valve1.Valve1 == 1]

    valve2 = data['harp_olfactometer'].streams.OdorValveState.data['Valve2'].reset_index()
    valve2.set_index('Time', inplace=True)
    valve2 = valve2.loc[valve2.Valve2 == 1]

    valve0['data']= np.where(valve0.Valve0 == 1, 'valve0', 0)
    valve1['data']= np.where(valve1.Valve1 == 1, 'valve1', 0)
    valve2['data']= np.where(valve2.Valve2 == 1, 'valve2', 0)

    valves = pd.concat([valve0['data'], valve1['data'], valve2['data']], axis=0).reset_index()

    new_df = pd.concat([endvalve, valves])
    new_df.set_index('Time', inplace=True)
    new_df.sort_index(inplace=True)

    value = 0
    for index, row in new_df.iterrows():
        if row['data'] != True:
            value = row['data'][-1:]
        else:
            row['data'] = int(value)
    return new_df, rig, odor_dict, duration, data_analog

In [ ]:
def plot_end_valves_and_save(new_df, rig, odor_dict, duration, data_analog, save_path, substract_baseline=False):
    window = (-1.5, duration+1.5)
    fig, ax = plt.subplots(1, 3, figsize=(18, 6), sharey=False)

    dataset1 = new_df.loc[new_df.data == 0].iloc[:-2]
    dataset2 = new_df.loc[new_df.data == 1].iloc[:-2]
    dataset3 = new_df.loc[new_df.data == 2].iloc[:-4]

    snippets = pd.DataFrame()
    for i, data_set in enumerate([dataset1, dataset2, dataset3]):
        trial_summary = pd.DataFrame()  
        count = 0
        for start_reward, row in data_set.iterrows():
            count += 1
            trial_average = pd.DataFrame()
            trial = data_analog.loc[start_reward + window[0]: start_reward + window[1], 'Channel0']
            
            if substract_baseline:
                trial_baseline = data_analog.loc[start_reward + window[0]: start_reward, 'Channel0'].mean()
                trial = trial - trial_baseline
                
            trial.index -= start_reward
            trial_average['data'] = trial.values
            trial_average['times'] = np.around(trial.index, 3)
            trial_average['visit'] = count

            trial_summary = pd.concat([trial_summary, trial_average], ignore_index=True)
        
        trial_summary['odor'] = i
        trial_summary['rig'] = rig
        sns.lineplot(data=trial_summary.loc[trial_summary.odor == i], x='times', y='data', hue='visit', ax=ax[i], palette='magma', ci='sd', alpha=0.6, legend=False, linewidth=0.5)
        
        snippets = pd.concat([snippets, trial_summary], ignore_index=True)
        ax[i].vlines(0, trial_summary.data.min(), trial_summary.data.max(), color='red', linestyle='--')
        ax[i].vlines(duration, trial_summary.data.min(), trial_summary.data.max(), color='indigo', linestyle='--')
        ax[i].set_title(f"{odor_dict[str(i)]}")
        ax[i].set_xlabel('Time from end valves (s)')

    sns.despine()
    plt.suptitle(f"End valve triggers for {rig} rig")
    plt.tight_layout()
    
    if substract_baseline:
        fig.savefig(save_path + f'\\calibration_substracted_{rig}_{odor_dict["0"]}_{odor_dict["1"]}_{odor_dict["2"]}.pdf', format='pdf', bbox_inches='tight')
    else:
        fig.savefig(save_path + f'\\calibration_{rig}_{odor_dict["0"]}_{odor_dict["1"]}_{odor_dict["2"]}.pdf', format='pdf', bbox_inches='tight')

    return snippets

In [ ]:
complete_session_path = r'Z:\scratch\vr-foraging\olfactometer_calibration\calibration_batch5_03122025\OlfactometerCalibration_2025-03-15T020111Z'
new_df, rig, odor_dict, duration, data_analog = recover_session(complete_session_path)
snippets = plot_end_valves_and_save(new_df, rig, odor_dict, duration, data_analog, save_path, substract_baseline=True)

In [ ]:
save_path = r'Z:\scratch\vr-foraging\olfactometer_calibration\calibration_batch5_generalization_06112025'
wholecalibration = pd.DataFrame()
for session in os.listdir(save_path):
    if session[-4:] == '.pdf' or session[-4:] == '.png':
        continue
    
    print(session)
    complete_session_path = save_path + '\\' + session
    new_df, rig, odor_dict, duration, data_analog = recover_session(complete_session_path)
    snippets = plot_end_valves_and_save(new_df, rig, odor_dict, duration, data_analog, save_path, substract_baseline=True)
    wholecalibration = pd.concat([wholecalibration, snippets], ignore_index=True)

In [ ]:
wholecalibration = wholecalibration.loc[wholecalibration.rig != '5B-test']
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=False)
for odor, ax in zip(wholecalibration.odor.unique(), axes.flatten()):
    sns.lineplot(data=wholecalibration.loc[wholecalibration.odor == odor], x='times', y='data', hue='rig', ci=None, alpha=0.6, legend=True, linewidth=0.5, ax=ax)
plt.legend(loc='upper right')
plt.suptitle(f"End valve triggers for all rigs")
plt.tight_layout()
sns.despine()
plt.savefig(save_path + f'\\calibration_all_rigs.pdf', format='pdf', bbox_inches='tight')

## Test with 5B rig to asses why it is lower than the other two

In [ ]:
wholecalibration = wholecalibration.loc[wholecalibration.rig != '5B-test']
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=False)
for odor, ax in zip(wholecalibration.odor.unique(), axes.flatten()):
    sns.lineplot(data=wholecalibration.loc[wholecalibration.odor == odor], x='times', y='data', hue='rig', ci=None, alpha=0.6, legend=True, linewidth=0.5, ax=ax)
plt.legend(loc='upper right')
plt.suptitle(f"End valve triggers for all rigs")
plt.tight_layout()
sns.despine()

In [ ]:
wholecalibration['rig'] = np.where((wholecalibration.odor == 2)&(wholecalibration.rig=='5B-test'), '5B - 4D vial', wholecalibration.rig)
wholecalibration['rig'] = np.where((wholecalibration.odor == 1)&(wholecalibration.rig=='5B-test'), '5B - second calibration', wholecalibration.rig)
wholecalibration['rig'] = np.where((wholecalibration.odor == 1)&(wholecalibration.rig=='5B'), '5B - first calibration', wholecalibration.rig)

fig, axes = plt.subplots(1, 1, figsize=(10, 5))

sns.lineplot(data=wholecalibration.loc[(wholecalibration.odor == 1)&(wholecalibration.rig=='4D')], x='times', y='data', hue='rig', ci=None, legend=True,  linewidth=1, ax=axes, color = 'black')

sns.lineplot(data=wholecalibration.loc[(wholecalibration.odor == 1)&(wholecalibration.rig=='5B - first calibration')], x='times', y='data', hue='rig', ci=None, legend=True, 
             linewidth=1, ax=axes, palette='Purples')
sns.lineplot(data=wholecalibration.loc[(wholecalibration.odor == 1)&(wholecalibration.rig=='5B - second calibration')], x='times', y='data', hue='rig', ci=None, legend=True, 
             linewidth=1, ax=axes,  palette='viridis')
sns.lineplot(data=wholecalibration.loc[(wholecalibration.odor == 2)&(wholecalibration.rig=='5B - 4D vial')], x='times', y='data', hue='rig', ci=None, legend=True, linewidth=1, ax=axes,  palette='magma')

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.suptitle(f"End valve triggers for all rigs")
plt.tight_layout()
sns.despine()